In [1]:
import os, re
import nltk, sklearn
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup as BS
import json
import numpy as np

In [2]:
blog2013_path = "G:\\author profiling\\pan13-author-profiling-training-corpus-2013-01-09\\en\\"
blog2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-blogs-2014-04-16\\"
review2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-reviews-2014-04-16\\"
social2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-socialmedia-2014-04-16\\"
twitter2015_path = "G:\\author profiling\\pan15-author-profiling-training-dataset-2015-04-23\\pan15-author-profiling-training-dataset-english-2015-04-23\\"
twitter2016_path = "G:\\author profiling\\pan16-author-profiling-training-dataset-2016-04-25\\pan16-author-profiling-training-dataset-english-2016-04-25\\"

In [3]:
#files = os.listdir(file_path)
blog2013_files = os.listdir(blog2013_path)
blog2014_files = os.listdir(blog2014_path)
review2014_files = os.listdir(review2014_path)
social2014_files = os.listdir(social2014_path)
twitter2015_files = os.listdir(twitter2015_path)
twitter2016_files = os.listdir(twitter2016_path)

In [4]:
if len(blog2013_files)>200:
    blog2013_files = np.random.choice(blog2013_files[:-1], 200)
    print "blog2013_files",len(blog2013_files)
else:
    blog2013_files = blog2013_files[:-1]
if len(blog2014_files)>200 :
    blog2014_files = np.random.choice(blog2014_files[:-1], 200)
    print "blog2014_files",len(blog2014_files)
else:
    blog2014_files = blog2014_files[:-1]
if len(review2014_files)>200:
    review2014_files = np.random.choice(review2014_files[:-1], 200)
    print "review2014_files",len(review2014_files)
else:
    review2014_files = review2014_files[:-1]
if len(social2014_files)>200:
    social2014_files = np.random.choice(social2014_files[:-1], 200)
    print "review2014_files", len(social2014_files)
else:
    social2014_files = social2014_files[:-1]
if len(twitter2015_files)>200:
    twitter2015_files = np.random.choice(twitter2015_files[:-1], 200)
    print "twitter2015_files", len(twitter2015_files)
else:
    twitter2015_files = twitter2015_files[:-1]
if len(twitter2016_files)>200:
    twitter2016_files = np.random.choice(twitter2016_files[:-1], 200)
    print "twitter2016_files", len(twitter2016_files)
else:
    twitter2016_files = twitter2016_files[:-1]

blog2013_files 200
review2014_files 200
review2014_files 200
twitter2016_files 200


In [150]:
from sklearn.feature_extraction.text import CountVectorizer
def find_MRC_POS(text_Series):
    #print text_Series
    vectorizer = CountVectorizer(max_df=0.65,min_df=5)
    X = vectorizer.fit_transform(text_Series)
    features = vectorizer.get_feature_names()
    features_POS = map(lambda X:X[1],nltk.pos_tag(features))
    MRC_POS = {'N':0,'J':0,'V':0,'A':0,'R':0,'C':0,'U':0,'I':0,'P':0,'O':0}
    nltk_POS = {'NN':'N', 'NNS':'N', 'NNP':'N', 'NNPS':'N', 'JJ':'J','JJR':'J','JJS':'J','MD':'V','WP':'U',
                'VB':'V', 'VBD':'V', 'VBG':'V', 'VBN':'V', 'VBP':'V', 'VBZ':'V','RB':'A','RBR':'A', 
                'RBS':'A', 'WRB':'A', 'IN':'R','RP':'R','CC':'C','TO':'C','PRP':'U', 'PRP$':'U','WP$':'U',
                'EX':'I', 'UH':'I', 'CD':'O', 'DT':'O', 'FW':'O', 'LS':'O', 'PDT':'O', 'POS':'O', 'WDT':'O'}
    features_MRC_POS = map(lambda x: nltk_POS[x], features_POS)
    text_POS = []
    for row in X.toarray():
        p = pd.Series(row)
        text_POS.append(' '.join([features_MRC_POS[i] for i in p[p>0].index.tolist()]))
#         print zip([features[i] for i in p[p>0].index.tolist()],[features_POS[i] for i in p[p>0].index.tolist()],[features_MRC_POS[i] for i in p[p>0].index.tolist()])
#         print sum(row)
#         break
# features_MRC_POS
    p = pd.Series(text_POS)
    #print text_POS
    return text_POS

# Blog 2013

In [151]:
count = 0
blog2013_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
for file in blog2013_files:
    #print file
    author = {}
    author['file'] = file
    with open(blog2013_path+file,'r') as f:
        tree = ET.parse(f)
        root = tree.getroot()
        author['gender'] = root.attrib['gender']
        author['age'] = root.attrib['age_group']
        #print gender, age
        raw_text = []
        for node in tree.iter('conversation'):
            #print node.tag, node.attrib
            #print node.text
            try:
                raw_text.append(BS(node.text, 'html.parser').get_text().strip())
            except:
                continue
        author['text'] = '\n'.join(raw_text)
    author_series = pd.Series(author)
    author_series.name = author['file']
#     author_series =  pd.Series(dict(zip(['file', 'text', 'gender', 'age'], [author['file'],author['text'],author['gender'],author['age']])))
    author_series.name = author['file']
    blog2013_df = blog2013_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count," Records processsed"
blog2013_df['MRC_POS'] = find_MRC_POS(blog2013_df['text'])        
blog2013_df.head()        

50  Records processsed
100  Records processsed
150  Records processsed
200  Records processsed


,file,text,gender,age,MRC_POS
4040cd1f2cdc51a847dfadfed9fb6e24_en_30s_female.xml,4040cd1f2cdc51a847dfadfed9fb6e24_en_30s_female...,Give and it wil come back to u.\nbe comfortabl...,female,30s,A J J V U
84202b89d4b6bf136998efd49e3c976b_en_30s_female.xml,84202b89d4b6bf136998efd49e3c976b_en_30s_female...,Any affordable resort in London that is inside...,female,30s,O O O N V V R J J N A O N A J J A V V J J N V ...
3e6dafa50b26745648707ec61e8278d0_en_30s_male.xml,3e6dafa50b26745648707ec61e8278d0_en_30s_male.xml,Canines are among the most common sorts of pet...,male,30s,V A N A V J A R J O O J N J A A V J N N N N C ...
9ba8b4c876ae609ec20d518de169dcfb_en_20s_male.xml,9ba8b4c876ae609ec20d518de169dcfb_en_20s_male.xml,It is important to report your injury as soon ...,male,20s,J R J V A R O O J J N N J J R J V V N R J J A ...
19b9cdf91fa54461c2c344831f3da2be_en_30s_male.xml,19b9cdf91fa54461c2c344831f3da2be_en_30s_male.xml,Decorating your home used to be really simple-...,male,30s,J R J N N N N N N N A V O O N N N A V N N J A ...


# Review 2014

In [153]:
review2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-reviews-2014-04-16\\"
truth_file = os.listdir(blog2014_path)[-1]
reviews_truth_df = pd.read_csv(review2014_path+truth_file, sep=":::", header=None, names=('file','gender','age'))
# print truth_df.columns.values
reviews_truth_df = reviews_truth_df.set_index('file')
review_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
for file in review2014_files:
    author = {}
    author['file'] = file[:-4]
    #print file
    with open(review2014_path+file) as f:
        tree = ET.parse(f)
        raw_text = []        
        for node in tree.iter('document'):
            raw_text.append(BS(node.text, 'html.parser').get_text().strip())
        author['text'] = '\n'.join(raw_text)
        author['gender'] = reviews_truth_df.loc[author['file']]['gender']
        #print author['gender']
        author['age'] = reviews_truth_df.loc[author['file']]['age']
    author_series = pd.Series(author)
    author_series.name = author['file']
#     author_series =  pd.Series(dict(zip(['file', 'text', 'gender', 'age'], [author['file'],author['text'],author['gender'],author['age']])))
    author_series.name = author['file']
    review_df = review_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count," records processed"
review_df['MRC_POS'] = find_MRC_POS(review_df['text'])
review_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


50  records processed
100  records processed
150  records processed
200  records processed


,file,text,gender,age,MRC_POS
a11889bb92d6969286194525e90bfae1,a11889bb92d6969286194525e90bfae1,Excellent Hotel for the right traveller! I rea...,FEMALE,35-49,J R A A N O N O V R R J A N N V J R N R V J N ...
ccec4a7f232815502c8d8474bf76b95a,ccec4a7f232815502c8d8474bf76b95a,Loved the Mayfair Hotel/Location We returned l...,FEMALE,65-xx,J O O V R R N A N N V A N A J N N V V V J R V ...
3a089701a1d2b65589b209fff22bc512,3a089701a1d2b65589b209fff22bc512,Wonderful hotel! Can't beat Iberostar when it ...,FEMALE,25-34,A A V O O V N R R A V N R V R V J C V N N V N ...
2ccbc2d25d9c2095fbd8f85954954c12,2ccbc2d25d9c2095fbd8f85954954c12,"For the most part,wonderful! We enjoyed Culebr...",MALE,50-64,N A O V N N J V V N J C J V N J V R J V V A J ...
4d392c7a6e253558c4068fa3ca121d76,4d392c7a6e253558c4068fa3ca121d76,I just don't get this hotel - So terrible I ju...,MALE,35-49,O O O O O O O R R O R A V J R N R A J J V N V ...


# Social Media

In [154]:
#social2014_path = "G:\\author profiling\\pan14-author-profiling-training-corpus-english-socialmedia-2014-04-16\\"
socialmedia_truth_df = pd.read_csv(social2014_path+os.listdir(social2014_path)[-1], sep=":::", header=None, names=('file','gender','age'))
socialmedia_truth_df = socialmedia_truth_df.set_index('file')
social_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
for file in social2014_files:
    author = {}
    author['file'] = file[:-4]
    #print file
    with open(social2014_path+file) as f:
        tree = ET.parse(f)
        raw_text = []        
        for node in tree.iter('document'):
            raw_text.append(BS(BS(node.text, 'html.parser').get_text().strip(),'html.parser').get_text())            
        author['text'] = '\n'.join(raw_text)
        author['gender'] = socialmedia_truth_df.loc[author['file']]['gender']
        #print author['gender']
        author['age'] = socialmedia_truth_df.loc[author['file']]['age']
    author_series = pd.Series(author)
    author_series.name = author['file']
    social_df = social_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count, " records processed"
social_df['MRC_POS'] = find_MRC_POS(social_df['text'])
social_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


50  records processed
100  records processed
150  records processed
200  records processed


,file,text,gender,age,MRC_POS
429653a8ee0a1220cb5f0fa67c2705f,429653a8ee0a1220cb5f0fa67c2705f,iphone repair TorontoIf you need to have a new...,FEMALE,35-49,O O O O A N N N N J V V N J N V V J A J V V V ...
e479736e602c1fc8bcc969b53ee37763,e479736e602c1fc8bcc969b53ee37763,On the internet Travel Portals - Generating Yo...,MALE,18-24,O O O O O O O O O O O N R A N N N J J J V A V ...
78dc1c79a97ad8772cc5d77e9fff676b,78dc1c79a97ad8772cc5d77e9fff676b,Starbucks to drop beetle juice from the menuSt...,MALE,25-34,O O O O O O O O O O O O O O O O O O O O O O O ...
3bb5355eac980b2c610dd5525d33e93c,3bb5355eac980b2c610dd5525d33e93c,homeopathy system of medicine explainedAs outl...,FEMALE,18-24,O O O A N V N J V N N V J N J N A V V J A N N ...
349bd6c48993d6b6d5e88b91003baefe,349bd6c48993d6b6d5e88b91003baefe,Sexy test> > This is the SEXY test.> > Post th...,MALE,18-24,O O O O O O O O O O O O O O O O O O O O O O O ...


# Twitter 2015

In [156]:
twitter2015_truth_df = pd.read_csv(twitter2015_path+os.listdir(twitter2015_path)[-1], sep=":::", header=None, names=('file','gender','age','p1','p2','p3','p4','p5'))
twitter2015_truth_df = twitter2015_truth_df.set_index('file')
twitter2015_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
for file in twitter2015_files:
    author = {}
    author['file'] = file[:-4]
    with open(twitter2015_path+file) as f:
        tree = ET.parse(f)
        raw_text = []        
        for node in tree.iter('document'):
            try:
                raw_text.append(node.text)
            except:
                continue
            #print raw_text.append(BS(BS(node.text, 'html.parser').get_text().strip(),'html.parser').get_text())            
        author['text'] = '\n'.join(raw_text)
        author['gender'] = twitter2015_truth_df.loc[author['file']]['gender']
        #print author['gender']
        author['age'] = twitter2015_truth_df.loc[author['file']]['age']
    #break
    author_series = pd.Series(author)
    author_series.name = author['file']
    twitter2015_df = twitter2015_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count, " records processed"
twitter2015_df['MRC_POS'] = find_MRC_POS(twitter2015_df['text'])
twitter2015_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


50  records processed
100  records processed
150  records processed


,file,text,gender,age,MRC_POS
02ae95de-7ee3-453a-978d-25d28b3f1a88,02ae95de-7ee3-453a-978d-25d28b3f1a88,Things I want for my business cards but are to...,M,25-34,O O O O O O O O O O O O O N A N N A R N N A A ...
03f72f70-7a00-4dbb-93b7-6c7f65954fc5,03f72f70-7a00-4dbb-93b7-6c7f65954fc5,"""If you fight for a cause then fight for a cau...",M,25-34,O O O O O O O O O O O O O N A V R A J A A A V ...
0aa7bace-924c-40fb-a2e5-3c7012ede244,0aa7bace-924c-40fb-a2e5-3c7012ede244,@username that tweet was mainly for you ;-) ch...,M,35-49,O O O O O O O O O J N N V V N N O N O A N N N ...
0b66092a-440e-4755-a624-759a580a1c70,0b66092a-440e-4755-a624-759a580a1c70,@username 'red' and 'blue' and then you wait t...,F,25-34,O O O O O O O O J N N N N A R N A A V V N N J ...
1094e809-1306-4cd1-8ac1-ebfdf7c872b8,1094e809-1306-4cd1-8ac1-ebfdf7c872b8,I sit with a boy in english he became my frien...,F,18-24,O O O O O O O N R N N A N A V J N O A V N R N ...


# Twitter 2016

In [157]:
twitter2016_truth_df = pd.read_csv(twitter2016_path+os.listdir(twitter2016_path)[-1], sep=":::", header=None, names=('file','gender','age'))
twitter2016_truth_df = twitter2016_truth_df.set_index('file')
twitter2016_df = pd.DataFrame(columns=('file', 'text', 'gender', 'age'))
count = 0
# print twitter2016_files
for file in twitter2016_files:
    author = {}
    author['file'] = file[:-4]
#     print author['file']
    with open(twitter2016_path+file) as f:
        try:
            tree = ET.parse(f)
            raw_text = []        
            for node in tree.iter('document'):
                raw_text.append(BS(node.text, 'html.parser').get_text().strip())
        except:
            continue
        author['text'] = '\n'.join(raw_text)
        author['gender'] = twitter2016_truth_df.loc[author['file']]['gender']
#         print author['text']
        author['age'] = twitter2016_truth_df.loc[author['file']]['age']
#     break
    author_series = pd.Series(author)
    author_series.name = author['file']
    twitter2016_df = twitter2016_df.append(author_series)
    count += 1
    if count%50 == 0:
        print count, " records processed"
twitter2016_df['MRC_POS'] = find_MRC_POS(twitter2016_df['text'])
twitter2016_df.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


50  records processed
100  records processed
150  records processed


,file,text,gender,age,MRC_POS
1e51e1f0ccf130be68b42fdaf201f40e,1e51e1f0ccf130be68b42fdaf201f40e,More great coverage of the #divestment campaig...,MALE,25-34,O O O O O O O O O O O O O O O O O O O O O O O ...
88d1f93972e3c4462b85bc2fa7a9eaae,88d1f93972e3c4462b85bc2fa7a9eaae,"@TomNixonSpeaks, sign up for Klout and see how...",MALE,50-64,O O O O O O N N J R V N J A N V A N N J A N N ...
3316270c2e911fc9ee491dbad386ed4e,3316270c2e911fc9ee491dbad386ed4e,Problemática del Analista es a Cuál Jugador Re...,MALE,65-xx,O O O O O O O O O O O O O O O O O O O O O O O ...
815c739d16821a8df42594567ea83862,815c739d16821a8df42594567ea83862,Nueva imagen de Batman vs Deathstroke en el ví...,MALE,35-49,O O O O O O O O O O O O O O O O O N N N N N N ...
11a689d54c4af2394bc91b4003e527f3,11a689d54c4af2394bc91b4003e527f3,"If you've been involved in an auto accident, y...",MALE,35-49,O O O O O O O O O O O O O O O O O O O O O O O ...


In [149]:
twitter2016_df['MRC_POS'] = find_MRC_POS(twitter2016_df['text'])
twitter2016_df.head()

['O O O O O O O O O O O O O O O O O O O O O O O J O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O R A J J J N N J V N R N V N N J J N J A V N N N N J N N J N A R N A V V N N N N N N N N V V A V A A J J A A R A N J R J N N J J N N N V J N V V V J O N V N O N N N V A V N V J V V N J N V N R V N N V V N N J A N N N N N A N N N J J J J V N A J V V J N N N V N N V N N N N N N N N N J N N N N N N A V J V A N N V N N V J V N J R V V A N N N N R J R N N V O V J N N V N N N O N N V N N J V O N J J N N V V N V R N N N V V V N J V N N J J J N N V N N V A N V J N N N N J J N V V N V N N V N N N N J N N N N N N N N N N V N N N N N N N N N V N V N V N N N N N A N N V V N V N V N N N N N N V V N N J V J V N N N J N N N N N N J N N N N N J N V A N N N N N N J N N N N N V N V N V V V N N J N N V N J N N N N N V A V N N N V N N N N J N V V N N N N V V N N V J V N J N N N N N N N N N V N V N V J V V J J N N V V V N N N J N J N N N J J A V J A N N V J J J N N N V N N N V A N N A N V N N N N 

,file,text,gender,age,MRC_POS
1e51e1f0ccf130be68b42fdaf201f40e,1e51e1f0ccf130be68b42fdaf201f40e,More great coverage of the #divestment campaig...,MALE,25-34,O O O O O O O O O O O O O O O O O O O O O O O ...
88d1f93972e3c4462b85bc2fa7a9eaae,88d1f93972e3c4462b85bc2fa7a9eaae,"@TomNixonSpeaks, sign up for Klout and see how...",MALE,50-64,O O O O O O N N J R V N J A N V A N N J A N N ...
3316270c2e911fc9ee491dbad386ed4e,3316270c2e911fc9ee491dbad386ed4e,Problemática del Analista es a Cuál Jugador Re...,MALE,65-xx,O O O O O O O O O O O O O O O O O O O O O O O ...
815c739d16821a8df42594567ea83862,815c739d16821a8df42594567ea83862,Nueva imagen de Batman vs Deathstroke en el ví...,MALE,35-49,O O O O O O O O O O O O O O O O O N N N N N N ...
11a689d54c4af2394bc91b4003e527f3,11a689d54c4af2394bc91b4003e527f3,"If you've been involved in an auto accident, y...",MALE,35-49,O O O O O O O O O O O O O O O O O O O O O O O ...


In [128]:
# vectorizer.fit_transform(blog2013_df['text'])
# print "blog2013: ",len(vectorizer.get_feature_names())
# vectorizer.fit_transform(social_df['text'])
# print "social2014: ",len(vectorizer.get_feature_names())
# vectorizer.fit_transform(review_df['text'])
# print "review2014: ",len(vectorizer.get_feature_names())
# vectorizer.fit_transform(twitter2015_df['text'])
# print "twitter2015: ",len(vectorizer.get_feature_names())
# X = vectorizer.fit_transform(twitter2016_df['text'])
# print "twitter2016: ",len(vectorizer.get_feature_names())

In [158]:
from sklearn import preprocessing

In [159]:
le = preprocessing.LabelEncoder()
le.fit(review_df['age'])
print list(le.classes_)
review_df['encoded_age'] = le.transform(review_df['age'])
le.fit(review_df['gender'])
print list(le.classes_)
review_df['encoded_gender'] = le.transform(review_df['gender'])
review_df.head()

['18-24', '25-34', '35-49', '50-64', '65-xx']
['FEMALE', 'MALE']


,file,text,gender,age,MRC_POS,encoded_age,encoded_gender
a11889bb92d6969286194525e90bfae1,a11889bb92d6969286194525e90bfae1,Excellent Hotel for the right traveller! I rea...,FEMALE,35-49,J R A A N O N O V R R J A N N V J R N R V J N ...,2,0
ccec4a7f232815502c8d8474bf76b95a,ccec4a7f232815502c8d8474bf76b95a,Loved the Mayfair Hotel/Location We returned l...,FEMALE,65-xx,J O O V R R N A N N V A N A J N N V V V J R V ...,4,0
3a089701a1d2b65589b209fff22bc512,3a089701a1d2b65589b209fff22bc512,Wonderful hotel! Can't beat Iberostar when it ...,FEMALE,25-34,A A V O O V N R R A V N R V R V J C V N N V N ...,1,0
2ccbc2d25d9c2095fbd8f85954954c12,2ccbc2d25d9c2095fbd8f85954954c12,"For the most part,wonderful! We enjoyed Culebr...",MALE,50-64,N A O V N N J V V N J C J V N J V R J V V A J ...,3,1
4d392c7a6e253558c4068fa3ca121d76,4d392c7a6e253558c4068fa3ca121d76,I just don't get this hotel - So terrible I ju...,MALE,35-49,O O O O O O O R R O R A V J R N R A J J V N V ...,2,1


In [160]:
le = preprocessing.LabelEncoder()
le.fit(social_df['age'])
print list(le.classes_)
social_df['encoded_age'] = le.transform(social_df['age'])
le.fit(social_df['gender'])
print list(le.classes_)
social_df['encoded_gender'] = le.transform(social_df['gender'])
social_df.head()

['18-24', '25-34', '35-49', '50-64', '65-xx']
['FEMALE', 'MALE']


,file,text,gender,age,MRC_POS,encoded_age,encoded_gender
429653a8ee0a1220cb5f0fa67c2705f,429653a8ee0a1220cb5f0fa67c2705f,iphone repair TorontoIf you need to have a new...,FEMALE,35-49,O O O O A N N N N J V V N J N V V J A J V V V ...,2,0
e479736e602c1fc8bcc969b53ee37763,e479736e602c1fc8bcc969b53ee37763,On the internet Travel Portals - Generating Yo...,MALE,18-24,O O O O O O O O O O O N R A N N N J J J V A V ...,0,1
78dc1c79a97ad8772cc5d77e9fff676b,78dc1c79a97ad8772cc5d77e9fff676b,Starbucks to drop beetle juice from the menuSt...,MALE,25-34,O O O O O O O O O O O O O O O O O O O O O O O ...,1,1
3bb5355eac980b2c610dd5525d33e93c,3bb5355eac980b2c610dd5525d33e93c,homeopathy system of medicine explainedAs outl...,FEMALE,18-24,O O O A N V N J V N N V J N J N A V V J A N N ...,0,0
349bd6c48993d6b6d5e88b91003baefe,349bd6c48993d6b6d5e88b91003baefe,Sexy test> > This is the SEXY test.> > Post th...,MALE,18-24,O O O O O O O O O O O O O O O O O O O O O O O ...,0,1


In [161]:
le = preprocessing.LabelEncoder()
le.fit(twitter2015_df['age'])
print list(le.classes_)
twitter2015_df['encoded_age'] = le.transform(twitter2015_df['age'])
le.fit(twitter2015_df['gender'])
print list(le.classes_)
twitter2015_df['encoded_gender'] = le.transform(twitter2015_df['gender'])
twitter2015_df.head()

['18-24', '25-34', '35-49', '50-XX']
['F', 'M']


,file,text,gender,age,MRC_POS,encoded_age,encoded_gender
02ae95de-7ee3-453a-978d-25d28b3f1a88,02ae95de-7ee3-453a-978d-25d28b3f1a88,Things I want for my business cards but are to...,M,25-34,O O O O O O O O O O O O O N A N N A R N N A A ...,1,1
03f72f70-7a00-4dbb-93b7-6c7f65954fc5,03f72f70-7a00-4dbb-93b7-6c7f65954fc5,"""If you fight for a cause then fight for a cau...",M,25-34,O O O O O O O O O O O O O N A V R A J A A A V ...,1,1
0aa7bace-924c-40fb-a2e5-3c7012ede244,0aa7bace-924c-40fb-a2e5-3c7012ede244,@username that tweet was mainly for you ;-) ch...,M,35-49,O O O O O O O O O J N N V V N N O N O A N N N ...,2,1
0b66092a-440e-4755-a624-759a580a1c70,0b66092a-440e-4755-a624-759a580a1c70,@username 'red' and 'blue' and then you wait t...,F,25-34,O O O O O O O O J N N N N A R N A A V V N N J ...,1,0
1094e809-1306-4cd1-8ac1-ebfdf7c872b8,1094e809-1306-4cd1-8ac1-ebfdf7c872b8,I sit with a boy in english he became my frien...,F,18-24,O O O O O O O N R N N A N A V J N O A V N R N ...,0,0


In [162]:
le = preprocessing.LabelEncoder()
le.fit(twitter2016_df['age'])
print list(le.classes_)
twitter2016_df['encoded_age'] = le.transform(twitter2016_df['age'])
le.fit(twitter2016_df['gender'])
print list(le.classes_)
twitter2016_df['encoded_gender'] = le.transform(twitter2016_df['gender'])
twitter2016_df.head()

['18-24', '25-34', '35-49', '50-64', '65-xx']
['FEMALE', 'MALE']


,file,text,gender,age,MRC_POS,encoded_age,encoded_gender
1e51e1f0ccf130be68b42fdaf201f40e,1e51e1f0ccf130be68b42fdaf201f40e,More great coverage of the #divestment campaig...,MALE,25-34,O O O O O O O O O O O O O O O O O O O O O O O ...,1,1
88d1f93972e3c4462b85bc2fa7a9eaae,88d1f93972e3c4462b85bc2fa7a9eaae,"@TomNixonSpeaks, sign up for Klout and see how...",MALE,50-64,O O O O O O N N J R V N J A N V A N N J A N N ...,3,1
3316270c2e911fc9ee491dbad386ed4e,3316270c2e911fc9ee491dbad386ed4e,Problemática del Analista es a Cuál Jugador Re...,MALE,65-xx,O O O O O O O O O O O O O O O O O O O O O O O ...,4,1
815c739d16821a8df42594567ea83862,815c739d16821a8df42594567ea83862,Nueva imagen de Batman vs Deathstroke en el ví...,MALE,35-49,O O O O O O O O O O O O O O O O O N N N N N N ...,2,1
11a689d54c4af2394bc91b4003e527f3,11a689d54c4af2394bc91b4003e527f3,"If you've been involved in an auto accident, y...",MALE,35-49,O O O O O O O O O O O O O O O O O O O O O O O ...,2,1


In [163]:
le = preprocessing.LabelEncoder()
le.fit(blog2013_df['age'])
print list(le.classes_)
blog2013_df['encoded_age'] = le.transform(blog2013_df['age'])
le.fit(blog2013_df['gender'])
print list(le.classes_)
blog2013_df['encoded_gender'] = le.transform(blog2013_df['gender'])
blog2013_df.head()

['10s', '20s', '30s']
['female', 'male']


,file,text,gender,age,MRC_POS,encoded_age,encoded_gender
4040cd1f2cdc51a847dfadfed9fb6e24_en_30s_female.xml,4040cd1f2cdc51a847dfadfed9fb6e24_en_30s_female...,Give and it wil come back to u.\nbe comfortabl...,female,30s,A J J V U,2,0
84202b89d4b6bf136998efd49e3c976b_en_30s_female.xml,84202b89d4b6bf136998efd49e3c976b_en_30s_female...,Any affordable resort in London that is inside...,female,30s,O O O N V V R J J N A O N A J J A V V J J N V ...,2,0
3e6dafa50b26745648707ec61e8278d0_en_30s_male.xml,3e6dafa50b26745648707ec61e8278d0_en_30s_male.xml,Canines are among the most common sorts of pet...,male,30s,V A N A V J A R J O O J N J A A V J N N N N C ...,2,1
9ba8b4c876ae609ec20d518de169dcfb_en_20s_male.xml,9ba8b4c876ae609ec20d518de169dcfb_en_20s_male.xml,It is important to report your injury as soon ...,male,20s,J R J V A R O O J J N N J J R J V V N R J J A ...,1,1
19b9cdf91fa54461c2c344831f3da2be_en_30s_male.xml,19b9cdf91fa54461c2c344831f3da2be_en_30s_male.xml,Decorating your home used to be really simple-...,male,30s,J R J N N N N N N N A V O O N N N A V N N J A ...,2,1


# Gender

In [203]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [225]:
print "Random Forest Classfier-gender"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier() ),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', RandomForestClassifier() ),
])
print "TfIdf"
pipeline_tfidf.fit(review_df['text'], review_df['encoded_gender'])
predicted = pipeline_tfidf.predict(social_df['text'])
print "train: review, test:social  -",accuracy_score(social_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: review, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

Random Forest Classfier-gender
TfIdf
train: review, test:social  - 0.55
train: review, test: twitter2015 - 0.552631578947
train: review, test: twitter2016 - 0.493670886076
train: review, test: blog - 0.505
POS
train: review, test: review - 0.505
train: review, test: twitter2015 - 0.5
train: review, test: twitter2016 - 0.525316455696
train: review, test: blog - 0.575


In [226]:
print "Multinomial Naive-Gender"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', MultinomialNB()),
])


print "TfIdf"
pipeline_tfidf.fit(review_df['text'], review_df['encoded_gender'])
predicted = pipeline_tfidf.predict(social_df['text'])
print "train: review, test:social  -",accuracy_score(social_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: review, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

Multinomial Naive-Gender
TfIdf
train: review, test:social  - 0.565
train: review, test: twitter2015 - 0.486842105263
train: review, test: twitter2016 - 0.5
train: review, test: blog - 0.515
POS
train: review, test: review - 0.505
train: review, test: twitter2015 - 0.5
train: review, test: twitter2016 - 0.525316455696
train: review, test: blog - 0.575


In [206]:
print "Logistic Regression-Gender"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='log')),
])
pipeline_pos = Pipeline([
    ('tfidf', TfidfVectorizer(use_idf= False,vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('clf', SGDClassifier(loss='log')),
])

print "TfIdf"
pipeline_tfidf.fit(review_df['text'], review_df['encoded_gender'])
predicted = pipeline_tfidf.predict(social_df['text'])
print "train: review, test:social  -",accuracy_score(social_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: review, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

Logistic Regression-Gender
TfIdf
train: review, test:social  - 0.53
train: review, test: twitter2015 - 0.506578947368
train: review, test: twitter2016 - 0.518987341772
train: review, test: blog - 0.48
POS
train: review, test: review - 0.505
train: review, test: twitter2015 - 0.5
train: review, test: twitter2016 - 0.525316455696
train: review, test: blog - 0.575


In [222]:
print "Logistic Regression-Gender"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='log')),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(max_df=0.0,vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', SGDClassifier(loss='log')),
])

print "TfIdf"
pipeline_tfidf.fit(social_df['text'], social_df['encoded_gender'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: social, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: social, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

Logistic Regression-Gender
TfIdf
train: social, test: review - 0.525
train: social, test: twitter2015 - 0.473684210526
train: social, test: twitter2016 - 0.481012658228
train: social, test: blog - 0.55
POS
train: social, test: review - 0.505
train: social, test: twitter2015 - 0.5
train: social, test: twitter2016 - 0.525316455696
train: social, test: blog - 0.575


In [221]:
print "Multinomial Naive-Gender"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', MultinomialNB()),
])

print "TfIdf"
pipeline_tfidf.fit(social_df['text'], social_df['encoded_gender'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: social, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: social, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

Multinomial Naive-Gender
TfIdf
train: social, test: review - 0.485
train: social, test: twitter2015 - 0.486842105263
train: social, test: twitter2016 - 0.53164556962
train: social, test: blog - 0.57
POS
train: social, test: review - 0.505
train: social, test: twitter2015 - 0.5
train: social, test: twitter2016 - 0.525316455696
train: social, test: blog - 0.575


In [220]:
print "Random Forest Classfier-gender"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier() ),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', RandomForestClassifier() ),
])

print "TfIdf"
pipeline_tfidf.fit(social_df['text'], social_df['encoded_gender'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: social, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: social, test: review -",accuracy_score(review_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_gender'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_gender'],predicted)

Random Forest Classfier-gender
TfIdf
train: social, test: review - 0.505
train: social, test: twitter2015 - 0.5
train: social, test: twitter2016 - 0.613924050633
train: social, test: blog - 0.57
POS
train: social, test: review - 0.505
train: social, test: twitter2015 - 0.5
train: social, test: twitter2016 - 0.525316455696
train: social, test: blog - 0.575


# Age

In [219]:
print "Random Forest Classfier-age"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier() ),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', RandomForestClassifier() ),
])

print "TfIdf"
pipeline_tfidf.fit(social_df['text'], social_df['encoded_age'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: social, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: social, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

Random Forest Classfier-age
TfIdf
train: social, test: review - 0.215
train: social, test: twitter2015 - 0.394736842105
train: social, test: twitter2016 - 0.23417721519
train: social, test: blog - 0.335
POS
train: social, test: review - 0.105
train: social, test: twitter2015 - 0.381578947368
train: social, test: twitter2016 - 0.0379746835443
train: social, test: blog - 0.09


In [218]:
print "Multinomial Naive"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', MultinomialNB()),
])

print "TfIdf"
pipeline_tfidf.fit(social_df['text'], social_df['encoded_age'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: social, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_gender'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: social, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

Multinomial Naive
TfIdf
train: social, test: review - 0.215
train: social, test: twitter2015 - 0.197368421053
train: social, test: twitter2016 - 0.373417721519
train: social, test: blog - 0.4
POS
train: social, test: review - 0.105
train: social, test: twitter2015 - 0.381578947368
train: social, test: twitter2016 - 0.0379746835443
train: social, test: blog - 0.09


In [217]:
print "Logistic Regression-age"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='log')),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', SGDClassifier(loss='log')),
])

print "TfIdf"
pipeline_tfidf.fit(social_df['text'], social_df['encoded_age'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: social, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_age'])
predicted = pipeline_pos.predict(review_df['text'])
print "train: social, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: social, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: social, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: social, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

Logistic Regression-age
TfIdf
train: social, test: review - 0.21
train: social, test: twitter2015 - 0.342105263158
train: social, test: twitter2016 - 0.322784810127
train: social, test: blog - 0.3
POS
train: social, test: review - 0.235
train: social, test: twitter2015 - 0.144736842105
train: social, test: twitter2016 - 0.430379746835
train: social, test: blog - 0.58


In [216]:
print "Logistic Regression-age"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05,ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='log')),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', SGDClassifier(loss='log')),
])

print "TfIdf"
pipeline_tfidf.fit(review_df['text'], review_df['encoded_age'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: review, test: social -",accuracy_score(social_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_age'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: review, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

Logistic Regression-age
TfIdf
train: review, test: social - 0.18
train: review, test: twitter2015 - 0.25
train: review, test: twitter2016 - 0.417721518987
train: review, test: blog - 0.215
POS
train: review, test: review - 0.235
train: review, test: twitter2015 - 0.144736842105
train: review, test: twitter2016 - 0.430379746835
train: review, test: blog - 0.58


In [215]:
print "Multinomial Naive-age"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', MultinomialNB()),
])

print "TfIdf"
pipeline_tfidf.fit(review_df['text'], review_df['encoded_age'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: review, test: social -",accuracy_score(social_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_age'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: review, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

Multinomial Naive-age
TfIdf
train: review, test: social - 0.225
train: review, test: twitter2015 - 0.171052631579
train: review, test: twitter2016 - 0.348101265823
train: review, test: blog - 0.23
POS
train: review, test: review - 0.215
train: review, test: twitter2015 - 0.394736842105
train: review, test: twitter2016 - 0.272151898734
train: review, test: blog - 0.33


In [214]:
print "Random Forest Classfier-age"
pipeline_tfidf = Pipeline([
    ('vect', CountVectorizer(max_df=0.05,min_df=0.05)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier() ),
])
pipeline_pos = Pipeline([
    ('vect', CountVectorizer(vocabulary = {'N':0,'J':1,'V':2,'A':3,'R':4,'C':5,'U':6,'I':7,'P':8,'O':9})),
    ('tfidf', TfidfTransformer(use_idf= False)),
    ('clf', RandomForestClassifier() ),
])
print "TfIdf"
pipeline_tfidf.fit(review_df['text'], review_df['encoded_age'])
predicted = pipeline_tfidf.predict(review_df['text'])
print "train: review, test: social -",accuracy_score(social_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2015_df['text'])
predicted = pipeline_tfidf.predict(twitter2015_df['text'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(twitter2016_df['text'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_tfidf.predict(blog2013_df['text'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

print "POS"
pipeline_pos.fit(social_df['MRC_POS'], social_df['encoded_age'])
predicted = pipeline_pos.predict(review_df['MRC_POS'])
print "train: review, test: review -",accuracy_score(review_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2015_df['MRC_POS'])
print "train: review, test: twitter2015 -",accuracy_score(twitter2015_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(twitter2016_df['MRC_POS'])
print "train: review, test: twitter2016 -",accuracy_score(twitter2016_df['encoded_age'],predicted)

predicted = pipeline_pos.predict(blog2013_df['MRC_POS'])
print "train: review, test: blog -",accuracy_score(blog2013_df['encoded_age'],predicted)

Random Forest Classfier-age
TfIdf
train: review, test: social - 0.2
train: review, test: twitter2015 - 0.243421052632
train: review, test: twitter2016 - 0.291139240506
train: review, test: blog - 0.26
POS
train: review, test: review - 0.235
train: review, test: twitter2015 - 0.144736842105
train: review, test: twitter2016 - 0.430379746835
train: review, test: blog - 0.58
